In [10]:
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv
import os
from time import sleep

load_dotenv()

True

In [11]:
client = OpenAI(
  api_key=os.getenv("OPENAI_API_KEY"),
)


In [7]:
df = pd.read_csv("2022-01_skynews_articles.csv", sep=';')

df.dropna(subset=['Tags'], inplace=True)
df_first = df.drop(df[df['Tags'] == "['N/A']"].index)
df_second = df_first.drop(df_first[df_first['Tags'] == "[]"].index)

df_second.to_csv("01_skynews_articles_clean.csv", sep=';')

In [17]:
df = pd.read_csv("2022_skynews_articles_clean_part5.csv", sep=';')
df = df.reset_index()
print(df.shape)

(82, 11)


In [18]:
def get_res(batch):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": "You will be provided with a batch of items. Each item consists of a title and one or multiple tags that both belong to one news article. The items start with <itemStart> and end with <itemEnd>. Your task is to classify the topic of a news article by looking at its title and tags in either politics, business and economy, environment, sports, entertainment and culture, science and technology or health. Return only the number of the Article and its category separated with a whitespace in a python list where the result matches the order of the input."},
        {"role": "user", "content": f"{batch}"}
        ],
        temperature=0,
        max_tokens=1000
    )
    categories = response.choices[0].message.content[1:-1].split(",")
    return categories

In [19]:
batchSize = 10

cat = {
     1: "politics",
     2: "business and economy",
     3: "environment",
     4: "sports",
     5: "entertainment and culture",
     6: "science and technology",
     7: "health"
    }

counter = 0
idx_lo = 0
idx_hi = 0
batch = ""
for index, row in df.iterrows():
    print(index)
    if counter < batchSize:
        title = row['Title']
        tags = row['Tags'][1:-1]
        counter += 1
        batch += f"<itemStart> ARTICLE-{counter}: Title: {title}; Tags: {tags} <itemEnd>"
    else:
        categories = get_res(batch)
        print(categories)
        if len(categories) < 10:
             break
        for idx_category, category in enumerate(categories):
            print(index + 1 - len(categories)+ idx_category, category)
            df.loc[index - len(categories) + idx_category, ['Chat GPT']] = category.strip().replace(",", "")
        sleep(22)
        title = row['Title']
        tags = row['Tags'][1:-1]
        counter = 1
        batch = f"<itemStart> ARTICLE-{counter}: Title: '{title}'; Tags: {tags}] <itemEnd>"

if len(batch) > 0:
    categories = get_res(batch)
    for idx_category, category in enumerate(categories):
            print(df.shape[0] - len(categories)+ idx_category, category)
            df.loc[df.shape[0] - len(categories) + idx_category, ['Chat GPT']] = category.strip().replace(",", "")


df.to_csv("2022_skynews_chatgpt_results_05.csv", sep=';')


0
1
2
3
4
5
6
7
8
9
10
["'ARTICLE-1: politics'", " 'ARTICLE-2: health'", " 'ARTICLE-3: politics'", " 'ARTICLE-4: health'", " 'ARTICLE-5: politics'", " 'ARTICLE-6: politics'", " 'ARTICLE-7: health'", " 'ARTICLE-8: politics'", " 'ARTICLE-9: politics'", " 'ARTICLE-10: politics'"]
1 'ARTICLE-1: politics'
2  'ARTICLE-2: health'
3  'ARTICLE-3: politics'
4  'ARTICLE-4: health'
5  'ARTICLE-5: politics'
6  'ARTICLE-6: politics'
7  'ARTICLE-7: health'
8  'ARTICLE-8: politics'
9  'ARTICLE-9: politics'
10  'ARTICLE-10: politics'


/var/folders/sh/fjb1r_5j6gxcy4lzfcc4_4zr0000gn/T/ipykernel_53131/679340383.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value ''ARTICLE-1: politics'' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index - len(categories) + idx_category, ['Chat GPT']] = category.strip().replace(",", "")


11
12
13
14
15
16
17
18
19
20
["'ARTICLE-1: politics'", " 'ARTICLE-2: politics'", " 'ARTICLE-3: politics'", " 'ARTICLE-4: politics'", " 'ARTICLE-5: politics'", " 'ARTICLE-6: entertainment and culture'", " 'ARTICLE-7: health'", " 'ARTICLE-8: environment'", " 'ARTICLE-9: politics'", " 'ARTICLE-10: politics'"]
11 'ARTICLE-1: politics'
12  'ARTICLE-2: politics'
13  'ARTICLE-3: politics'
14  'ARTICLE-4: politics'
15  'ARTICLE-5: politics'
16  'ARTICLE-6: entertainment and culture'
17  'ARTICLE-7: health'
18  'ARTICLE-8: environment'
19  'ARTICLE-9: politics'
20  'ARTICLE-10: politics'
21
22
23
24
25
26
27
28
29
30
["'ARTICLE-1: politics'", " 'ARTICLE-2: health'", " 'ARTICLE-3: politics'", " 'ARTICLE-4: politics'", " 'ARTICLE-5: politics'", " 'ARTICLE-6: politics'", " 'ARTICLE-7: politics'", " 'ARTICLE-8: crime'", " 'ARTICLE-9: business and economy'", " 'ARTICLE-10: politics'"]
21 'ARTICLE-1: politics'
22  'ARTICLE-2: health'
23  'ARTICLE-3: politics'
24  'ARTICLE-4: politics'
25  'ARTICLE-5

In [16]:
df.to_csv("2022_skynews_chatgpt_results_05.csv", sep=';')

In [22]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": "Generate a high quality text about a provided topic which is the size of 2,500 words"},
        {"role": "user", "content": f"Title: Fun"}
        ],
        temperature=0,
        max_tokens=4000
    )
category = response.choices[0].message.content
print(response)
print(category)


KeyboardInterrupt: 

In [42]:
counter = 0
for index, row in df.iterrows():
    print(counter)
    title = row['Title']
    tags = row['Tags'][1:-1]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": "You will be provided with a title and one or multiple tags of a news article. Your task is to classify to which of the following topics the article belongs to. The topics to choose from are: 'politics', 'business and economy', 'environment', 'sports', 'entertainment and culture', 'science and technology' or 'health'. Return only one of the provided topics and keep the same format as they are provided."},
        {"role": "user", "content": f"Title: '{title}'; Tags: {tags}"}
        ],
        temperature=0,
        max_tokens=256
    )
    category = response.choices[0].message.content
    df.loc[index, ['Chat GPT']] = category
    sleep(5)
    counter += 1


df.to_csv("2022_skynews_chatgpt_results_03.csv", sep=';')

NameError: name 'response' is not defined